# Connect to Existing Chrome Browser

This example shows how to connect webtask to a customer's existing Chrome browser.

## Customer Setup

Before running this notebook, launch Chrome with debugging enabled:

```bash
# On Mac/Windows:
chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation

# On Linux:
google-chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation
```

**Important**: Chrome requires `--user-data-dir` for security when using remote debugging. This creates a separate profile that persists your login sessions between runs.

## Requirements

- Chrome/Chromium launched with `--remote-debugging-port=9222` and `--user-data-dir`
- `GOOGLE_API_KEY` environment variable set

In [1]:
# Imports
import asyncio
import os
from webtask import Webtask
from webtask.integrations.llm import GeminiLLM
from webtask.integrations.browser.playwright import PlaywrightBrowser

/home/stevewang/Github/web-auto/webtask/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
async def main():
    # Connect to customer's existing Chrome browser
    print("📡 Connecting to existing Chrome browser...")
    browser = await PlaywrightBrowser.connect("http://localhost:9222")
    print("✅ Connected!")

    # Create LLM
    api_key = os.getenv("GOOGLE_API_KEY")
    llm = GeminiLLM.create(model="gemini-2.5-flash", api_key=api_key)

    # Create Webtask and agent using the connected browser
    wt = Webtask()
    agent = await wt.create_agent_with_browser(
        llm=llm,
        browser=browser,
        use_screenshot=True
    )

    print("🤖 Agent ready! Controlling customer's browser...")

    # Example: Navigate to a page in customer's browser
    await agent.navigate("https://example.com")
    await agent.wait_for_load()

    print("🎯 Task complete!")

    # Cleanup: Close the Playwright connection (but not the customer's browser!)
    # This disconnects from CDP but leaves the browser running
    await browser.close()

    print("👋 Disconnected. Customer's browser stays open.")

In [3]:
# Run the example
await main()

📡 Connecting to existing Chrome browser...
✅ Connected!
🤖 Agent ready! Controlling customer's browser...
🎯 Task complete!
👋 Disconnected. Customer's browser stays open.
